# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [1]:
from pprint import pprint
from pathlib import Path

import pandas as pd

from wombat.core import Simulation, Metrics
from wombat.core.library import load_yaml

pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Get the path and file name pattern from the simulation file naming convention
events_fname = Path(sim.env.events_log_fname)

# Get the path to the file which is one level up from the logging outputs
fpath = events_fname.parents[1]

# Get the filename and update it to the correct name
fname = events_fname.name
fname = fname.replace("_events.log", "_metrics_inputs.yaml")

# Load the metrics data
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 95.5%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.98,0.98,0.98,0.97,0.98,0.98,0.95,0.97,0.96,0.98,0.98,0.96,0.96,0.92,0.98,0.98,0.92,0.98,0.88,0.97,0.85,0.99,0.98,0.96,0.98,0.94,0.98,0.98,0.99,0.98,0.98,0.98,0.98,0.96,0.98,0.98,0.93,0.87,0.98,0.87,0.88,0.97,0.98,0.87,0.98,0.96,0.98,0.98,0.98,0.94,0.98,0.98,0.98,0.93,0.98,0.98,0.90,0.95,0.86,0.94,0.98,0.97,0.98,0.92,0.91,0.98,0.98,0.89,0.98,0.97,0.98,0.95,0.95,0.98,0.90,0.88,0.98,0.98,0.91,0.98


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.98
2004,0.94
2005,0.97
2006,0.97
2007,0.95
2008,0.94
2009,0.91
2010,0.94
2011,0.98


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.96
2,0.95
3,0.95
4,0.94
5,0.94
6,0.95
7,0.96
8,0.96
9,0.96


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.99
     2          0.98
     3          0.97
     4          0.96
     5          0.97
     6          0.99
     7          0.99
     8          0.99
     9          0.99
     10         0.99
     11         0.97
     12         0.95
2004 1          0.95
     2          0.93
     3          0.92
     4          0.91
     5          0.93
     6          0.93
     7          0.95
     8          0.96
     9          0.95
     10         0.97
     11         0.97
     12         0.97

## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.46%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.46,0.47,0.47,0.46,0.47,0.46,0.46,0.46,0.46,0.47,0.47,0.46,0.46,0.45,0.46,0.47,0.44,0.47,0.42,0.46,0.41,0.47,0.47,0.46,0.47,0.45,0.47,0.47,0.47,0.47,0.47,0.47,0.47,0.46,0.47,0.47,0.43,0.42,0.46,0.42,0.42,0.47,0.47,0.42,0.47,0.46,0.47,0.47,0.47,0.44,0.47,0.47,0.47,0.44,0.47,0.46,0.43,0.45,0.41,0.44,0.47,0.46,0.47,0.43,0.43,0.47,0.46,0.42,0.47,0.46,0.47,0.45,0.45,0.47,0.43,0.42,0.47,0.47,0.43,0.47


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.44
2005,0.47
2006,0.45
2007,0.49
2008,0.49
2009,0.43
2010,0.40
2011,0.49


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.60
2,0.50
3,0.50
4,0.40
5,0.38
6,0.32
7,0.33
8,0.36
9,0.46


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.63
     2          0.37
     3          0.37
     4          0.52
     5          0.34
     6          0.31
     7          0.30
     8          0.30
     9          0.28
     10         0.43
     11         0.54
     12         0.54
2004 1          0.53
     2          0.49
     3          0.49
     4          0.37
     5          0.28
     6          0.36
     7          0.26
     8          0.40
     9          0.56
     10         0.55
     11         0.48
     12         0.49

## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 100%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,0.98
2004,0.99
2005,1.02
2006,1.00
2007,0.99
2008,1.01
2009,1.00
2010,1.00
2011,0.98


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,0.95
2,1.02
3,1.02
4,1.00
5,1.03
6,1.04
7,1.03
8,0.98
9,1.00


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

Completion Rate
year month                 
2003 1                 0.90
     2                 1.06
     3                 0.96
     4                 0.99
     5                 1.06
     6                 1.00
     7                 0.99
     8                 0.98
     9                 1.02
     10                0.97
     11                1.01
     12                0.83
2004 1                 0.69
     2                 0.81
     3                 1.43
     4                 1.04
     5                 1.33
     6                 1.11
     7                 1.12
     8                 0.96
     9                 1.01
     10                0.84
     11                1.10
     12                0.82

## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $508,752.99/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","2,660,000.00","100,260,937.50"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"11,372,843.75"
2004,"13,031,250.00"
2005,"10,504,125.00"
2006,"11,836,937.50"
2007,"13,232,250.00"
2008,"11,392,187.50"
2009,"11,727,562.50"
2010,"12,049,437.50"
2011,"12,463,500.00"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"543,083.33","543,083.33","543,083.33",0.00,0.00
2,"496,052.08","496,052.08","496,052.08",0.00,0.00
3,"541,989.58","541,625.00","541,989.58",0.00,0.00
4,"524,635.42","525,000.00","524,635.42",0.00,0.00
5,"542,500.00","542,500.00","542,500.00",0.00,"32,484,375.00"
6,"525,000.00","525,000.00","525,000.00","2,660,000.00",0.00
7,"542,500.00","542,500.00","542,500.00",0.00,"31,979,687.50"
8,"542,500.00","542,500.00","542,500.00",0.00,0.00
9,"525,000.00","525,000.00","525,000.00",0.00,"35,345,312.50"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   54,760.42               54,760.42   
     2                   49,000.00               49,000.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   
2004 1                   54,250.00               54,250.00   
     2                   50,750.00               50,750.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   54,760.42                  0.00               0.00  
     2                   49,000.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00       3,689,062.50  
     6                   52,500.00            266,000.00               0.00  
     7                   54,250.00                  0.00       2,750,000.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00       2,750,000.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00               0.00  
2004 1                   54,250.00                  0.00               0.00  
     2                   50,750.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00       3,373,437.50  
     6                   52,500.00            266,000.00               0.00  
     7                   54,250.00                  0.00       3,218,750.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00       4,251,562.50  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00               0.00

## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,0.75,0.75,0.75,0.29,0.54


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,0.60,0.63,0.62,0.10,0.53
2004,0.88,0.87,0.86,0.45,0.62
2005,0.86,0.85,0.87,0.28,0.36
2006,0.81,0.77,0.76,0.31,0.67
2007,0.75,0.76,0.75,0.45,0.60
2008,0.75,0.75,0.73,0.28,0.64
2009,0.72,0.74,0.75,0.24,0.56
2010,0.73,0.72,0.75,0.34,0.64
2011,0.69,0.68,0.71,0.10,0.52


## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [24]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [25]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [26]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [27]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [28]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00
2004 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
##### **NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [29]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,208,190.10"
Repair,"21,062,781.25"
Crew Transfer,"1,578,645.84"
Site Travel,0.00
Mobilization,"15,000,000.00"
Weather Delay,"31,938,059.90"
No Requests,"19,977,296.88"
Not in Shift,"30,335,744.79"


In [30]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,208,190.10","2,208,190.10"
Repair,0.00,0.00,0.00,"21,062,781.25","21,062,781.25"
Crew Transfer,0.00,0.00,0.00,"1,578,645.84","1,578,645.84"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"15,000,000.00","15,000,000.00"
Weather Delay,0.00,0.00,0.00,"31,938,059.90","31,938,059.90"
No Requests,0.00,0.00,0.00,"19,977,296.88","19,977,296.88"
Not in Shift,0.00,0.00,0.00,"30,335,744.79","30,335,744.79"


In [31]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

total_cost
year reason                    
2003 Maintenance      51,752.60
     Repair        1,896,625.00
     Crew Transfer   130,484.38
     Site Travel           0.00
     Mobilization  1,500,000.00
     Weather Delay 1,628,541.67
     No Requests   3,991,390.63
     Not in Shift  2,174,049.48
2004 Maintenance     415,205.73
     Repair        2,550,135.42

In [32]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

total_cost
month reason                   
1     Maintenance    112,966.15
      Repair         241,973.96
      Crew Transfer   36,804.69
      Site Travel          0.00
      Mobilization         0.00
      Weather Delay  226,078.12
      No Requests    698,815.10
      Not in Shift   312,611.98
2     Maintenance    135,916.67
      Repair         223,179.69

In [33]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          20,307.29
           Crew Transfer    2,406.25
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   18,666.67
           No Requests    116,994.79
           Not in Shift     5,906.25
     2     Maintenance          0.00
           Repair          22,111.98
           Crew Transfer    2,807.29
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   11,593.75
           No Requests    102,466.15
           Not in Shift     8,020.83
     3     Maintenance          0.00
           Repair          20,653.65
           Crew Transfer    2,515.63
           Site Travel          0.00

## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
##### **NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [34]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
generator,"123,248,140.63"


In [35]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
generator delay                  0.00              0.00   62,092,023.44   
          maintenance            0.00              0.00    2,208,190.10   
          repair                 0.00              0.00   21,062,781.25   

                         total_cost  
component action                     
generator delay       62,092,023.44  
          maintenance  2,208,190.10  
          repair      21,062,781.25

In [36]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,generator,"2,351,500.00",0.00,"5,872,046.88","8,223,546.88"
2004,generator,"5,083,000.00",0.00,"11,136,645.83","16,219,645.83"
2005,generator,"3,536,000.00",0.00,"4,655,695.31","8,191,695.31"
2006,generator,"4,199,000.00",0.00,"9,310,645.83","13,509,645.83"
2007,generator,"4,250,000.00",0.00,"10,803,023.44","15,053,023.44"
2008,generator,"3,759,500.00",0.00,"8,865,057.29","12,624,557.29"
2009,generator,"3,212,500.00",0.00,"9,016,322.92","12,228,822.92"
2010,generator,"4,161,000.00",0.00,"9,458,320.31","13,619,320.31"
2011,generator,"3,322,000.00",0.00,"9,755,505.21","13,077,505.21"


In [37]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,generator,"1,055,500.00",0.00,"921,028.65","1,976,528.65"
2,generator,"1,068,000.00",0.00,"861,966.15","1,929,966.15"
3,generator,"1,123,500.00",0.00,"918,458.33","2,041,958.33"
4,generator,"1,313,500.00",0.00,"856,315.10","2,169,815.10"
5,generator,"9,384,500.00",0.00,"28,467,468.75","37,851,968.75"
6,generator,"3,828,500.00",0.00,"1,669,098.96","5,497,598.96"
7,generator,"7,731,500.00",0.00,"22,938,606.77","30,670,106.77"
8,generator,"1,292,000.00",0.00,"811,489.58","2,103,489.58"
9,generator,"5,604,500.00",0.00,"26,187,221.35","31,791,721.35"


In [38]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     generator       22,000.00              0.00       44,880.21   
     2     generator       49,000.00              0.00       44,533.85   
     3     generator       49,000.00              0.00       33,742.19   
     4     generator       22,000.00              0.00       36,932.29   
     5     generator    1,067,500.00              0.00    3,233,687.50   
     6     generator      100,000.00              0.00       44,130.21   
     7     generator      357,500.00              0.00    1,120,567.71   
     8     generator       19,000.00              0.00       38,190.10   
     9     generator      385,500.00              0.00      998,718.75   
     10    generator       22,000.00              0.00       53,848.96   
     11    generator      127,000.00              0.00       79,861.98   
     12    generator      131,000.00              0.00      142,953.13   
2004 1     generator      229,500.00              0.00      161,875.00   
     2     generator      197,000.00              0.00      150,500.00   
     3     generator      222,500.00              0.00      161,000.00   
     4     generator      237,000.00              0.00      157,500.00   
     5     generator    1,280,500.00              0.00    3,033,562.50   
     6     generator      651,000.00              0.00      274,187.50   
     7     generator      776,500.00              0.00    2,789,059.90   
     8     generator      108,500.00              0.00       63,473.96   
     9     generator      868,000.00              0.00    3,905,945.31   
     10    generator      179,500.00              0.00      123,411.46   
     11    generator      168,000.00              0.00      153,380.21   
     12    generator      165,000.00              0.00      162,750.00   
2005 1     generator      233,500.00              0.00      160,125.00   
     2     generator      133,000.00              0.00      146,125.00   
     3     generator      177,500.00              0.00      161,000.00   
     4     generator      201,000.00              0.00       82,869.79   

                       total_cost  
year month component               
2003 1     generator    66,880.21  
     2     generator    93,533.85  
     3     generator    82,742.19  
     4     generator    58,932.29  
     5     generator 4,301,187.50  
     6     generator   144,130.21  
     7     generator 1,478,067.71  
     8     generator    57,190.10  
     9     generator 1,384,218.75  
     10    generator    75,848.96  
     11    generator   206,861.98  
     12    generator   273,953.13  
2004 1     generator   391,375.00  
     2     generator   347,500.00  
     3     generator   383,500.00  
     4     generator   394,500.00  
     5     generator 4,314,062.50  
     6     generator   925,187.50  
     7     generator 3,565,559.90  
     8     generator   171,973.96  
     9     generator 4,773,945.31  
     10    generator   302,911.46  
     11    generator   321,380.21  
     12    generator   327,750.00  
2005 1     generator   393,625.00  
     2     generator   279,125.00  
     3     generator   338,500.00  
     4     generator   283,869.79

## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [39]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [40]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,000,080.00"


In [41]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [42]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [43]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [44]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"222,917.65","72,906.25","66,023.75",507.00
major repair,"100,599.77","1,785.00","1,261.50",30.00
major replacement,"223,962.10","12,043.50","223,962.10",57.00
manual reset,"190,001.48","36,345.75","13,446.50","5,687.00"
medium repair,"12,569.56","9,927.50","8,014.75",205.00
minor repair,"65,125.65","31,173.00","15,479.75","2,216.00"


## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [45]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,590,818,242.50"


In [46]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,590,818,242.50","121,674,423.50","123,174,618.50","123,099,942.50","121,628,787.50","122,608,275.50","122,035,391.50","119,892,050.50","122,243,148.00","120,828,449.00","123,455,532.00","122,735,918.50","120,472,769.00","120,250,576.50","117,102,176.50","122,092,730.00","122,655,240.50","116,822,612.00","122,533,354.50","110,442,106.50","121,929,466.00","108,256,473.50","124,660,851.00","123,865,577.00","121,483,879.00","123,292,000.00","118,220,066.00","123,682,974.50","122,802,620.50","124,176,242.00","122,881,020.00","123,416,917.00","123,334,922.00","123,228,923.00","121,315,334.50","122,742,041.00","123,033,276.00","113,954,047.50","109,265,856.50","122,092,801.50","109,263,365.50","110,946,632.00","122,310,765.50","122,913,659.00","110,125,577.00","123,281,024.00","121,750,756.00","122,806,442.00","122,906,682.50","122,988,200.50","117,009,503.50","122,945,988.50","122,474,393.50","123,086,809.50","115,546,847.50","122,818,957.00","122,119,532.50","113,414,568.50","118,146,765.50","107,753,623.00","116,363,973.00","123,306,742.00","121,602,761.00","122,743,264.00","113,288,713.00","114,075,345.50","123,970,252.50","122,029,307.50","111,555,091.50","123,123,575.50","121,635,820.00","122,851,584.00","119,267,265.50","118,359,179.00","122,356,872.00","112,873,030.50","111,543,205.50","122,938,377.00","123,344,025.00","113,034,254.00","122,562,120.50"


In [47]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"864,578,487.00"
2004,"926,057,195.50"
2005,"997,585,084.50"
2006,"954,201,341.00"
2007,"1,022,594,000.50"
2008,"1,031,678,604.50"
2009,"906,202,824.50"
2010,"838,507,726.00"
2011,"1,035,500,351.50"


In [48]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,073,376,617.50"
2,"815,496,521.00"
3,"886,201,348.50"
4,"695,164,118.00"
5,"674,181,476.00"
6,"549,020,132.00"
7,"584,316,701.00"
8,"640,782,144.00"
9,"792,170,012.00"


In [49]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     112,190,181.00
     2      58,899,105.50
     3      66,418,627.50
     4      89,718,901.50
     5      61,085,323.50
     6      53,672,653.00
     7      53,814,143.50
     8      53,705,269.00
     9      47,668,682.00
     10     77,561,668.00
     11     93,724,325.50
     12     96,119,607.00
2004 1      94,650,815.00
     2      82,327,871.00
     3      86,970,030.50
     4      63,782,887.00
     5      49,871,125.00
     6      63,007,528.50
     7      47,305,564.00
     8      71,905,138.00
     9      97,595,194.00
     10     98,739,451.00
     11     82,514,346.00
     12     87,387,245.50

## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [50]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [51]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [52]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [53]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [54]:
SAM_settings = "SAM_Singleowner_defaults.yaml"
metrics.sam_settings = load_yaml(sim.env.data_dir / "windfarm", SAM_settings)
metrics._setup_pysam()

In [55]:
metrics.pysam_all_outputs()

,Value
Metric,
After Tax NPV ($),"33,792,859.30"
Nominal LCOE ($/kW),-0.01
Real LCOE ($/kW),-0.00
After Tax IRR (%),14.60


In [56]:
sim.env.cleanup_log_files(log_only=False)